In [78]:
import cv2
import os
import numpy as np

Following subjects are the names of person whose detection will be done

In [79]:
subjects = ["Unknown","Pruthviraj","Dhoni"]

detect_face function detects only one person's face
Cascade classifier is used for face detection
the following xml file 'lbpcascade_frontalface.xml' must be at same location of the notebook

In [80]:
def detect_face(img):
    #convert to gray Scale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #initiate classifier
    face_cascade = cv2.CascadeClassifier('lbpcascade_frontalface.xml')
    #detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.2, minNeighbors = 5)
    if(len(faces)==0):
        return None,None
    
    (x,y,w,h) = faces[0]
    return gray[y:y+w, x:x+h], faces[0]


detect_faces function detects multiple faces in a frame

In [81]:
def detect_faces(img):
    #convert to gray Scale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #initiate classifier
    face_cascade = cv2.CascadeClassifier('lbpcascade_frontalface.xml')
    #detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.2, minNeighbors = 5)
    
    if(len(faces)==0):
        return None,None
    
    multiple_faces = []
    rects = []
    for (x,y,w,h) in faces:
        gray_face = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        multiple_faces.append(gray_face[y:y+w, x:x+h])
        rects.append((x,y,w,h))
        
    return multiple_faces,rects

Here we prepare data to be trained. In the location there are folders starting with 'p' character
we can use any character this will help to specify that the folder contains training data and not any other data

In [82]:
def prepare_training_data(path):
    dirs = os.listdir(path)
    faces= []
    labels = []
    
    for dir_name in dirs:
        #check if directory name starts with s to idenitify persons data
        if not dir_name.startswith("p"):
            continue
        
        label = int(dir_name.replace("p",""))
        
        subj_dir_path = path + '\\'+ dir_name
        
        subj_image_names = os.listdir(subj_dir_path)
        
        for image_name in subj_image_names:
            if image_name.startswith("."):
                continue
            
            image_path = subj_dir_path + "\\"+image_name
            image = cv2.imread(image_path)
            
            cv2.imshow("training image...", image)
            cv2.waitKey(100)
            
            face,rect = detect_face(image)
            
            if face is not None:
                faces.append(face)
                labels.append(label)
                
    
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    
    return faces, labels
    

In [83]:
#Prepare train the data
faces,labels = prepare_training_data("Training-Data")
print("total faces: ", len(faces))
print("total labels: ",len(labels))

total faces:  35
total labels:  35


Train the data with LBPH trainer

In [84]:
#initiate the face recognizer
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.train(faces,np.array(labels))


Draws rectangle on image

In [85]:
#to draw rectangle on image
def draw_rectangle(img,rect):
    (x,y,w,h) = rect
    cv2.rectangle(img, (x,y),(x+w,y+h),(0,255,0),2)

Draws text on the image

In [86]:
#to put text on image
def draw_text(img,text,x,y):
    cv2.putText(img,text,(x,y),cv2.FONT_HERSHEY_PLAIN, 1.5, (0,255,0),2)

This function predicts single face in the frame
face recognizer returns the number of data set which the face belongs and match probability with 0 being the best match

In [87]:
#to predict single face
def predict(test_img):
    img = test_img.copy()
    label_text = subjects[0]
    
    face,rect = detect_face(img)
    
    if(face is not None):
        label = face_recognizer.predict(face)
    
        if(label[0] == 1  and label[1] < 50 ):
            label_text = subjects[1]
        elif(label[0] == 2 and label[1] < 50):
            label_text = subjects[2]
    
        draw_rectangle(img,rect)
    
        draw_text(img, label_text,rect[0],rect[1]-5)
    
        return img
    else:
        return None

This function predicts multiple faces in the frame

In [88]:
#To predict multiple faces
def predict_multiple_faces(test_img):
    img = test_img.copy()
    label_text = subjects[0]
    
    faces,rects = detect_faces(img)
    
    if faces is None:
        return None
    else:
        for i in range(len(faces)):
            label = face_recognizer.predict(faces[i])
            
            if(label[0] == 1  and label[1] < 50 ):
                label_text = subjects[1]
            elif(label[0] == 2 and label[1] < 50):
                label_text = subjects[2]
            
            draw_rectangle(img,rects[i])
    
            draw_text(img, label_text,rects[i][0],rects[i][1]-5)
        
        return img
        

This loops run each frame from web cam and feeds to predictor and shows the output with rectangle and recognition of face. Press escape to quit

In [90]:
cap = cv2.VideoCapture(0)
# loop runs if capturing has been initialized. 
while 1:  
   
    # reads frames from a camera 
    ret, img = cap.read() 
    
    predict_img = predict_multiple_faces(img)
    
    if(predict_img is None):
        cv2.imshow('Face Detection',img) 
    else:
        cv2.imshow('Face Detection',predict_img) 
  
    # Wait for Esc key to stop 
    k = cv2.waitKey(30) & 0xff
    if k == 27: 
        break
# Close the window 
cap.release() 
  
# De-allocate any associated memory usage 
cv2.destroyAllWindows()